In [ ]:
#default_exp uvis

In [ ]:
#export
from planetarypy.pds.opusapi import OPUS
from planetarypy.utils import url_retrieve, have_internet
from planetarypy.config import config
from yarl import URL
import tomlkit
from fastcore.utils import Path

In [ ]:
#export
storage_root = config.storage_root / "cassini/uvis"
storage_root

Path('/home/maye/big_drive/planetary_data/cassini/uvis')

In [ ]:
#export 
def get_user_guide():
    url = URL("https://pds-rings.seti.org/cassini/uvis/1-UVIS_Users_Guide_-2018-Jan%2015-For%20PDS-REV-2018-07-06.pdf")
    local_path = storage_root / "uvis_user_guide.pdf"
    if not local_path.exists():
        url_retrieve(url, storage_root / "uvis_user_guide.pdf")
    return local_path

In [ ]:
get_user_guide()

uvis_user_guide.pdf
:   0%|          | 0/19934688 [00:00<?, ?it/s]

Path('/home/maye/big_drive/planetary_data/cassini/uvis/uvis_user_guide.pdf')

In [ ]:
# export
class DataManager:
    def __init__(self, pid):
        self.pid = pid
        if not self.raw_data_path.exists():
            self.download()

    def query_pid(self, pid):
        opus = OPUS()
        self.query_result = opus.query_image_id(pid)[0]
        self.opus_id = self.query_result[0]
        self.dict = self.query_result[1]

    @property
    def pds_id(self):
        return self.pid[:17]

    @property
    def folder(self):
        #         return storage_root / "/".join(self.raw_data_url.parts[4:7])
        return storage_root / self.pds_id

    @property
    def raw_data_url(self):
        return URL(self.dict["couvis_raw"][0])

    @property
    def raw_label_url(self):
        return URL(self.dict["couvis_raw"][1])

    @property
    def raw_data_path(self):
        return self.folder / (self.pds_id + ".DAT")

    @property
    def raw_label_path(self):
        return self.raw_data_path.with_suffix(".LBL")

    @property
    def calib_corr_path(self):
        return self.raw_data_path.with_name(self.raw_data_path.stem + "_CAL_3.DAT")

    @property
    def original_pid_file(self):
        return self.folder / "original_pid.txt"

    @property
    def results_file(self):
        return self.folder / "urls.toml"

    def download(self, overwrite=False):
        if self.raw_data_path.exists() and not overwrite:
            print("Local files exists. Use `overwrite=True` to download fresh.")
            return
        self.query_pid(self.pds_id)
        self.original_pid_file.mk_write(self.pid)
        self.results_file.mk_write(tomlkit.dumps(self.dict))
        self.raw_data_path.parent.mkdir(parents=True, exist_ok=True)
        for key in ["couvis_raw", "couvis_calib_corr"]:
            for url in self.dict[key]:
                url_retrieve(url, self.folder / URL(url).name)

    def __repr__(self):
        s = f"Product ID:\n{self.id}\n\n"
        for k, v in self.query_result[1].items():
            s += f"Key: {k},\nValue(s):\n{v}\n\n"
        return s

In [ ]:
pid = "EUV2002_198_03_26_54_UVIS_C33ST_SPICARAST002_PRIME"

In [ ]:
pid = "EUV2005_172_03_35"

In [ ]:
dm = DataManager(pid)

Found 1 obsids.


EUV2005_172_03_35.DAT
:   0%|          | 0/21495808 [00:00<?, ?it/s]

EUV2005_172_03_35.LBL
:   0%|          | 0/4839 [00:00<?, ?it/s]

EUV2005_172_03_35_CAL_3.DAT
:   0%|          | 0/262144 [00:00<?, ?it/s]

EUV2005_172_03_35_CAL_3.LBL
:   0%|          | 0/13772 [00:00<?, ?it/s]

In [ ]:
dm.raw_data_path.exists()

False

In [ ]:
assert dm.pid == pid

PDS data is only stored under the first 17 characters of any UVIS-internal product id.

In [ ]:
assert dm.pds_id == pid[:17]

The originally used product ID `pid` will be stored under `original_pid.txt` in the data folder:

In [ ]:
dm.original_pid_file

Path('/home/maye/big_drive/planetary_data/cassini/uvis/EUV2002_198_03_26/original_pid.txt')

Also the OPUS result file will be stored as a TOML file:

In [ ]:
dm.results_file

Path('/home/maye/big_drive/planetary_data/cassini/uvis/EUV2002_198_03_26/urls.toml')

In [ ]:
dm.download(overwrite=False)

Local files exists. Use `overwrite=True` to download fresh.


In [ ]:
dm.raw_data_path

Path('/home/maye/big_drive/planetary_data/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26.DAT')

In [ ]:
dm.calib_corr_path

Path('/home/maye/big_drive/planetary_data/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26_CAL_3.DAT')

In [ ]:
#export
def get_data_path(pid):
    dm = DataManager(pid)
    return dm.raw_data_path

def get_label_path(pid):
    dm = DataManager(pid)
    return dm.raw_label_path

In [ ]:
get_data_path(pid)

Path('/home/maye/big_drive/planetary_data/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26.DAT')

In [ ]:
get_label_path(pid)

Path('/home/maye/big_drive/planetary_data/cassini/uvis/EUV2002_198_03_26/EUV2002_198_03_26.LBL')